# Entregable 2: Modelo Dimensional y Diseño Analítico



## 1. Introducción

Este documento presenta el diseño lógico y conceptual de un modelo dimensional para análisis del negocio e-commerce, basado en la metodología de Ralph Kimball. Se parte de una base de datos relacional previamente explorada ([Exploración-inicial](/pi_mod_02/delivery/01-first-delivery/1st_delivery.ipynb), [Informe-técnico](/pi_mod_02/delivery/01-first-delivery/1st_tech_report.ipynb)), a partir de la cual se identifican hechos, dimensiones y se propone un modelo estrella optimizado para responder a las preguntas de negocio.

---

## 2. Modelo relacional actual

### 2.1. Descripción general

El modelo relacional está compuesto por 12 tablas interrelacionadas que representan las principales entidades y procesos de un sistema de comercio electrónico. Si bien no se cuenta con una descripción formal del propósito del sistema, la estructura de los datos permite inferir que se trata de una plataforma de e-commerce que gestiona:

- Registro de usuarios
- Productos organizados por categoría
- Carritos de compras
- Órdenes y detalles de pedidos (compras)
- Métodos y registros de pagos
- Direcciones de envío de los usuarios
- Reseñas de productos

Estas entidades se modelan bajo un enfoque operacional, optimizado para soportar transacciones típicas de un retail: compras, pagos, reseñas, etc.


### 2.2. Estructura

Este modelo fue diseñado con un propósito transaccional , es decir, orientado a capturar y almacenar eventos del negocio en tiempo real con eficiencia y consistencia. Las tablas cuentan con claves primarias y foráneas para garantizar integridad referencial.

Principales relaciones del modelo:

- Cada usuario puede tener múltiples órdenes, carritos y direcciones de envío.

- Cada orden puede estar compuesta por varios productos (DetalleOrdenes).

- Los pagos se registran tanto en un historial (HistorialPagos) como en una tabla que permite múltiples métodos de pago por orden (OrdenesMetodosPago).

- Las reseñas vinculan usuarios con productos, permitiendo calificaciones y comentarios.


### 2.3. Relación con el modelo analítico

El modelo relacional sirve como fuente de datos para construir un modelo analítico bajo el enfoque dimensional. Sin embargo, su estructura requiere optimización para consultas analíticas complejas como:

- Ranking de productos por calificación o volumen vendido
- Evolución y distribución de ingresos por método de pago
- Total de ventas por categoría y mes


Para ello, se organizará un esquema , diferenciando entre:

- __Hechos__ (ventas, pagos, interacciones)

- __Dimensiones__ (producto, cliente, tiempo, categoría, etc.)

Este modelo relacional será la base para derivar dichos elementos. Las claves foráneas bien definidas y los atributos categóricos disponibles permitirán construir dimensiones completas, mientras que los montos, cantidades y fechas registradas se convertiran en métricas temporales y de negocio.


### 2.4. Diagrama Entidad-Relación


A continuación, se presenta el diagrama del modelo relacional actual de la base de datos:



![Diagrama ER](../../app/assets/ecommerce_der.svg)


--- 

## 3. Modelo lógico: hechos y dimensiones (Kimball)


### 3.1. Enfoque metodológico

El abordaje del aálisis de las preguntas de negocio se realizó según la metodología de Ralph Kimball. Este enfoque organiza la información en torno a tablas de hechos, que contienen métricas cuantificables, y tablas de dimensiones, que describen el contexto de esas métricas.

### 3.2. Identificación de hechos y dimensiones


Tabla de hechos principal: **FactVentas** (Tabla `Ordenes`)

Descripción: Esta tabla centraliza información relacionada a ventas (órdenes concretadas), asociando montos, cantidades, fechas y métodos de pago.

| Campo          | Descripción                              |
| -------------- | ---------------------------------------- |
| OrdenID        | Clave primaria (FK desde Ordenes)        |
| UsuarioID      | Cliente que realizó la orden             |
| FechaOrden     | Fecha de la compra                       |
| ProductoID     | Producto comprado (desde DetalleOrdenes) |
| Cantidad       | Unidades compradas                       |
| PrecioUnitario | Precio al momento de la venta            |
| MontoTotal     | Cantidad \* PrecioUnitario               |
| MetodoPagoID   | Medio de pago utilizado                  |


Otras posibles tablas de hechos:
*FactPagos*: enfocada en pagos registrados (Tabla `HistorialPagos`)
*FactReseñas*: para análisis de feedback de usuarios (Tabla `ReseñasProducto`)
*FactCarrito*: análisis de productos agregados pero no comprados (aunque la falta de relación entre carrito y ordenes, impide otros análisis más interesantes como conversión, tiempo de conversión, etc)


### 3.3. Tablas de dimensiones

| Dimensión        | Origen                 | Atributos clave                               |
| ---------------- | ---------------------- | --------------------------------------------- |
| `DimUsuarios`    | Usuarios               | Nombre, Apellido, Email, FechaRegistro        |
| `DimProductos`   | Productos + Categorías | Nombre, Descripción, Precio, Stock, Categoría |
| `DimCategorias`  | Categorías             | Nombre, Descripción                           |
| `DimFecha`       | Derivada               | Año, Mes, Día, DíaSemana, Trimestre, etc.     |
| `DimMetodosPago` | MétodosPago            | Nombre, Descripción                           |
| `DimUbicacion`   | DireccionesEnvio       | Provincia, Ciudad, País, etc.                 |

### 3.4 


#### <u> Tabla de hechos principal: **FactVentas** </u>

* Se presenta modelo propuesto para la tabla de hechos principal *FactVentas*


![Diagrama Fact Ventas](../../app/assets/ecommerce_fact_ventas.svg)


**Relación con preguntas de negocio**

| Pregunta de negocio                      |  Tabla de hechos                   | Dimensiones involucradas         | Notas                                           |
| ---------------------------------------- | ---------------------------------- | -------------------------------- | ----------------------------------------------- |
| Productos con alto stock y bajas ventas  | `FactVentas`                       | `DimProductos`                   | Stock en `DimProductos`, ventas en `FactVentas` |
| Categoría con mayor valor vendido        | `FactVentas`                       | `DimProductos` → `DimCategorias` | Suma de `Cantidad * PrecioUnitario`             |
| Usuarios con más de una orden            | `FactVentas`                       | `DimUsuarios`                    | Conteo de órdenes                               |
| Usuarios que más gastaron                | `FactVentas`                       | `DimUsuarios`                    | Suma de `MontoTotal` por usuario                |
| Órdenes por mes y su variación           | `FactVentas`                       | `DimFecha`                       | Conteo mensual, cálculo de variación            |
| Monto recaudado por mes                  | `FactVentas`                       | `DimFecha`                       | Suma por mes                                    |
| Productos más vendidos por volumen       | `FactVentas`                       | `DimProductos`                   | Suma de `Cantidad`                              |
| Ticket promedio por orden                | `FactVentas`                       | —                                | Promedio de `MontoTotal`                        |
| Categorías con más productos vendidos    | `FactVentas`                       | `DimProductos`, `DimCategorias`  | Suma `Cantidad` por categoría                   |
| Día de la semana con más ventas          | `FactVentas`                       | `DimFecha`                       | Filtro por `DiaSemana`                          |


**Justificación del modelo propuesto**


El modelo lógico propuesto se basa en el enfoque dimensional de Ralph Kimball.En este sentido, se identificó a la tabla FactVentas como la tabla de hechos principal, ya que concentra las métricas clave relacionadas con las operaciones de ventas concretadas: cantidades vendidas, montos involucrados, precios al momento de la transacción y medios de pago utilizados. Esta tabla actúa como núcleo del modelo, y se conecta con múltiples dimensiones que contextualizan cada acción, permitiendo así responder interrogantes desde múltiples perspectivas.

Las tablas de dimensiones aportan descripciones y jerarquías para la exploración de los datos. Por ejemplo:

- La dimensión productos (DimProductos) permite agrupar y filtrar por atributos como nombre, categoría o stock, fundamentales para analizar qué productos tienen alta o baja rotación.

- La dimensión fecha (DimFecha) facilita la exploración temporal, soportando análisis por año, mes, día, trimestre o día de la semana.

- La dimensión usuarios (DimUsuarios) habilita el análisis del comportamiento de los clientes, incluyendo su ubicación (a través de DimUbicacion) y fecha de registro.

- La dimensión método de pago permite desglosar ingresos por canal financiero.

Estrategia SDC : Se aplica SCD Tipo 2 en: 
    - DimUbicacion para preservar el historial de cambios en la localización del usuario, permitiendo así analizar eventos (como ventas o pagos) en su contexto geográfico original. 
    - DimCategorias también se modela como SCD Tipo 2, dado que un producto podría reclasificarse con el tiempo.

El modelo fue diseñado con especial atención a la posibilidad de escalar y extenderse. Por ello, se contemplan tablas de hechos secundarias como FactPagos, FactReseñas y eventualmente FactCarrito, las cuales podrían alimentar análisis más específicos como satisfacción del cliente, estado de pagos o comportamiento previo a la compra.

La estructura en estrella resultante, con FactVentas en el centro y las dimensiones enlazadas, permite responder de forma directa y eficiente a las preguntas de negocio definidas, tal como se muestra en la sección anterior. Además, la incorporación de jerarquías geográficas (país → provincia → ciudad) en la dimensión de ubicación mejora la calidad de los análisis territoriales y evita la dispersión de datos debida a campos libres o mal normalizados.


#### <u> Tabla de hechos secundaria: **FactPagos** </u>


**Estructura de la tabla FactPagos**


| Campo        | Descripción                                  |
| ------------ | -------------------------------------------- |
| PagoID       | Clave primaria (desde `HistorialPagos`)      |
| OrdenID      | Relación con la orden correspondiente        |
| UsuarioID    | Usuario que realizó el pago                  |
| MetodoPagoID | Método de pago utilizado                     |
| FechaPago    | Fecha efectiva del pago                      |
| MontoPagado  | Monto registrado en ese pago                 |
| EstadoPago   | Estado (Exitoso, Rechazado, Pendiente, etc.) |


**Relación con preguntas de negocio**


| Pregunta de negocio                         | Tabla de hechos | Dimensiones involucradas        | Notas                                           |
| ------------------------------------------- | --------------- | ------------------------------- | ----------------------------------------------- |
| Evolución de ingresos según fecha de pago   | `FactPagos`     | `DimFecha`                      | Fecha del pago                                  |
| Métodos de pago más utilizados              | `FactPagos`     | `DimMetodosPago`                | Suma de montos pagados según método             |
| Monto promedio por método de pago           | `FactPagos`     | `DimMetodosPago`                | Promedio de `Monto`                             |
| Monto total pagado por usuario              | `FactPagos`     | `DimUsuarios`                   | Agregación por usuario                          |
| Órdenes con múltiples pagos                 | `FactPagos`     | `DimUsuarios`, `DimMetodosPago` | Agrupamiento por OrdenID y conteo de registros  |


**Justificación del modelo propuesto**


La tabla FactPagos complementa la información de FactVentas al enfocarse en los pagos efectivamente realizados, independientemente del estado de la orden. Esto permitiría analizar aspectos clave del proceso de compra, como métodos de pago preferidos, tiempos entre orden y pago, pagos rechazados o parciales, contribuyendp a una visión financiera más detallada del negocio.

Se eligió utilizar `HistorialPagos` como fuente porque permite registrar múltiples eventos de pago por orden, facilitando el análisis de situaciones como reintentos o pagos fallidos. El campo EstadoPago, aunque categórico, se incluye como atributo de hecho para permitir análisis por tipo de evento. Alternativamente, podría normalizarse como una dimensión auxiliar.

Al igual que en FactVentas, esta tabla se enlaza con las dimensiones DimUsuarios, DimFecha, y DimMetodosPago, permitiendo desgloses por cliente, tiempo y canal financiero. También se conecta con OrdenID, lo cual facilita cruzar pagos con ventas en análisis más avanzados, por ejemplo: monto pagado vs monto facturado.


#### <u> Tabla de hechos secundaria: **FactReseñas** </u>

**Estructura de la tabla FactReseñas**


|  Campo       | Descripción                                |
| ----------- | ------------------------------------------ |
| ReseñaID    | Clave primaria (desde `ReseñasProducto`)   |
| UsuarioID   | Usuario que escribió la reseña             |
| ProductoID  | Producto evaluado                          |
| FechaReseña | Fecha en la que se registró la reseña      |
| Puntuacion  | Calificación numérica (por ejemplo, 1 a 5) |
| Comentario  | Texto libre con la opinión del usuario     |


**Relación con preguntas de negocio**


| Pregunta de negocio                                   | Tabla de hechos             | Dimensiones involucradas         | Notas                                           |
| ----------------------------------------------------- | --------------------------- | -------------------------------- | ----------------------------------------------- |
| Productos mejor calificados                           | `FactReseñas`               | `DimProductos`                   | Promedio de puntuación por producto             |
| Usuarios más participativos en reseñas                | `FactReseñas`               | `DimUsuarios`                    | Conteo de reseñas por usuario                   |
| Relación entre calificación y volumen de ventas       | `FactReseñas`, `FactVentas` | `DimProductos`                   | Requiere cruces entre hechos                    |
| Productos con baja calificación en períodos recientes | `FactReseñas`               | `DimProductos`, `DimFecha`       | Filtro temporal y por score                     |
| Distribución de calificaciones por categoría          | `FactReseñas`               | `DimProductos` → `DimCategorias` | Agrupamiento por categoría desde `DimProductos` |


**Justificación del modelo propuesto**


La tabla `FactReseñas` permite conocer la percepción del cliente. A diferencia de FactVentas, que refleja comportamiento transaccional, esta tabla recoge feedback cualitativo y cuantitativo, permitiendo evaluar satisfacción y reputación de productos.

Se seleccionó como fuente la tabla ReseñasProducto, que ya contiene todos los atributos necesarios. Si bien el campo Comentario es texto libre y no es útil para agregaciones directas, puede emplearse en análisis más avanzados con tecnología adicional.

Las dimensiones asociadas —usuario, producto y fecha— permiten desgloses detallados, como:

- Seguimiento temporal de la evolución de la satisfacción.

- Identificación de productos o categorías con problemas de calidad.

- Detección de patrones de comportamiento.

Este diseño puede ser enriquecido a futuro con métricas adicionales, como longitud del comentario, presencia de palabras clave, o puntuación normalizada.


A continuación se muestra modelo final con todas las tablas de hechos propuestas


![Diagrama Fact Ventas](../../app/assets/ecommerce_full_fact_tables.svg)



### 3.5. Consideraciones sobre Slowly Changing Dimensions (SCDs)


En el diseño de un modelo dimensional es fundamental definir cómo se gestionarán los cambios en los atributos de las dimensiones a lo largo del tiempo (SCD, por sus siglas en inglés).

- **DimUsuarios**: se podría aplicar SCD tipo 2 para registrar cambios en datos personales relevantes (email, ubicación).
- **DimProductos**: no se espera mucha variación, pero el precio podría justificar SCD tipo 2 si se requiere seguimiento histórico.
- **DimCategorias** : si un producto cambia de categoría, es útil mantener el historial para evitar distorsiones en reportes históricos.

Para este modelo se proponen distintas estrategias según la naturaleza de la dimensión:

➤ SCD Tipo 1 – Actualización simple (sin conservar historial)
- Se aplicará en dimensiones donde los cambios representan correcciones o donde no resulta relevante conservar la versión anterior de los datos. Esto implica que la actualización sobrescribe el valor anterior directamente.
Por ejemplo:

    - *DimProductos*: cambios en el nombre o descripción del producto.
    - *DimUsuarios*: corrección de errores tipográficos en nombre, email o apellido.

➤ SCD Tipo 2 – Conservación de historial de versiones
Se aplicará cuando sea importante preservar el estado histórico de una dimensión, para garantizar que los hechos reflejen correctamente el contexto temporal en el que ocurrieron.

- Dimensiones candidatas a esta estrategia:
    - DimUbicacion: si un usuario cambia de ciudad, conservar el lugar donde vivía al momento de una compra puede ser relevante para análisis geográficos históricos.
    - DimCategorias: si un producto cambia de categoría, es útil mantener el historial para evitar distorsiones en reportes históricos.

Para implementar SCD Tipo 2, se agregarían los siguientes campos a la dimensión:

FechaInicio: indica cuándo entra en vigencia esa versión del registro.

FechaFin: indica hasta cuándo fue válida esa versión (puede ser NULL si está activa).

EsActual: booleano que marca si la versión está activa.



** En esta fase no se implementaron SCDs, pero se recomienda su inclusión si el análisis lo justifica.
`UsuarioID`**


### 4. Conclusión


El modelo dimensional propuesto se diseñó con el objetivo de optimizar el análisis de un sistema de e-commerce, partiendo de un modelo relacional transaccional. Siguiendo la metodología Kimball, se identificaron hechos clave, dimensiones relevantes y se organizaron en una estructura en estrella que permite responder eficientemente a preguntas de negocio concretas, previamente analizadas en la [entrega 1](/pi_mod_02/delivery/01-first-delivery/1st_tech_report.ipynb).

La tabla de hechos principal (`FactVentas`) fue complementada con hechos secundarios (`FactPagos`, `FactReseñas`), lo que habilita un enfoque más amplio: desde ventas e ingresos hasta satisfacción y comportamiento de usuarios.

Asimismo, se propuso una estrategia de Slowly Changing Dimensions (SCD) adecuada al caso: Tipo 1 para cambios correctivos menores y Tipo 2 para atributos con impacto analítico histórico. Aunque no se implementó materialmente esta estrategia en esta fase, su diseño fue contemplado en las dimensiones correspondientes.


> Este proyecto me resultó valioso a nivel académico, al ofrecer la oportunidad de aplicar herramientas y conceptos propios de la ingeniería de datos a un caso realista, integrando el diseño lógico, el análisis de negocio y las decisiones técnicas en un único proceso de trabajo. Fue también un buen espacio para ensayar la toma de decisiones propias del modelado dimensional, valorando la simplicidad sin perder de vista la potencia analítica que se busca en un sistema de business intelligence.


In [4]:
!dbt debug --profiles-dir /app/.dbt --project-dir /app/dbt_project
!dbt run --profiles-dir /app/.dbt --project-dir /app/dbt_project


19:50:10  Running with dbt=1.10.5
19:50:10  dbt version: 1.10.5
19:50:10  python version: 3.11.13
19:50:10  python path: /usr/local/bin/python3.11
19:50:10  os info: Linux-6.8.0-64-generic-x86_64-with-glibc2.36
19:50:10  Using profiles dir at /app/.dbt
19:50:10  Using profiles.yml file at /app/.dbt/profiles.yml
19:50:10  Using dbt_project.yml file at /app/dbt_project/dbt_project.yml
19:50:10  adapter type: postgres
19:50:10  adapter version: 1.9.0
19:50:10  Configuration:
19:50:10    profiles.yml file [OK found and valid]
19:50:10    dbt_project.yml file [OK found and valid]
19:50:10  Required dependencies:
19:50:10   - git [ERROR]

19:50:10  Connection:
19:50:10    host: database
19:50:10    port: 5432
19:50:10    user: lu
19:50:10    database: ecommerce_db
19:50:10    schema: public
19:50:10    connect_timeout: 10
19:50:10    role: None
19:50:10    search_path: None
19:50:10    keepalives_idle: 0
19:50:10    sslmode: None
19:50:10    sslcert: None
19:50:10    sslkey: None
19:50:10   